In [10]:
import pandas as pd
from pprint import pprint
df = pd.read_csv("pericias_medicas.csv", sep=",", encoding="utf-8")
df = df['text'].apply(str)
df1 = df.head(20)

In [139]:
import re

def limpiarTexto(text):
    text = re.sub(r'[-,;+º*?¿!¡<>{}// %&´""]',' ',text)
    text = re.sub('^ +',' ',text)
    text = re.sub('\n ','',text)
    text = re.sub('\n\n+','\n',text)
    text = re.sub(' +\n','\n',text)
    text = re.sub('\. \n','\.\n',text)
    text = re.sub('\d\n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('¨[... ]','',text)
    text = text.lower()
    text = re.sub(' {2,100}',' ',text)
    text = re.sub('^ +','',text)
    text = re.sub('n.º.','',text)
    text = re.sub('[a-zA-z-.]@[a-zA-Z].com','',text)
    text = re.sub('\d+','',text)
    text = re.sub(r'^ ix|iv|vi+|i+x |(i{2,3})','', text)

    text = re.sub(' +\n+','\n',text)
    text = re.sub('\n+','\n',text)
    text = re.sub('\d+\n','',text)
    text.strip()
    return text

dfLimpio = df1.apply(limpiarTexto)

In [ ]:
#pprint(dfLimpio[0])

## Limpieza de texto

In [5]:
import nltk

stops = nltk.corpus.stopwords.words('spanish')

#elimino stopwords
def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return texto

df1.apply(str)
df1 = df1.apply(remove_stops)


In [6]:
#pprint(df1[10])

## Expresion regular para encontrar los titulos

In [157]:
#creo un nuevo data frame de titulos
titulos1 = (re.findall(r'\n[a-z- \.]+:\n', df1[0]))

def buscarTitulos(text):
    
    text = re.sub(r'^ ','', text)
    titulosBuscados = re.findall(r'\n[a-z- \.]+:\n', text)
    return titulosBuscados

def limpiarTitulos(text):
    text = re.sub(r'\\n +','\n', text)
    text = re.sub(r' +',' ', text)
    text = re.findall(r'[a-z ]+:', text)
    #text = re.sub(r'\\n','', text)
    return text

def sacarEspacios(text):
    text = re.sub(r' +',' ', text)
    return text

def limpio(text):
    text = re.sub(r'\\n +','\n', text)
    text = re.sub(r' +',' ', text)
    text = re.sub(r'^ ','', text)
    return text

dfTitulos = dfLimpio.apply(buscarTitulos)
dfTitulos = dfTitulos.apply(str)
dfTitulos = dfTitulos.apply(limpiarTitulos)
dfTitulos = dfTitulos.apply(str)
dfLimpio = dfLimpio.apply(limpio)
#dfTitulos = dfTitulos.apply(sacarEspacios)
pprint(dfTitulos[0])
#HACER ESTADISTICAS DE TITULOS


("[' juez nacional:', 'examen fisico del actor:', 'semiologia de las "
 "afecciones:', ' tobillo:', 'consideraciones medico legales:', "
 "'npsicotrauma:', 'npetitorio:']")


In [189]:
#lista de titulos y ubicacion
dfTitulos = dfTitulos.apply(str)
titulos = (re.findall(r'[ a-z-]+:', dfTitulos[0]))
titulosPosicion=[]
for i,j in enumerate(titulos):
    pprint(f'{i} , {j}')
    pprint(dfLimpio[0].index(titulos[i]))
    inicio = dfLimpio[0].index(titulos[i])
    pprint(inicio + (len(titulos[i])))
    final = inicio + (len(titulos[i]))
    pprint(final-inicio)
    #palabrasPorTitulo = final-inicio
    #titulosPosicion.append((titulos[i], inicio,final, palabrasPorTitulo))
pprint(titulosPosicion)
#anotar parrafos entre titulos.
#parrafo = texto[inicio:inicio+1]

'0 ,  juez nacional:'
176
191
15
'1 , examen fisico del actor:'
2084
2108
24
'2 , semiologia de las afecciones:'
5055
5084
29
'3 ,  tobillo:'
5475
5484
9
'4 , consideraciones medico legales:'
9326
9357
31
'5 , npsicotrauma:'


ValueError: substring not found

In [180]:
pprint(dfLimpio[0].index(titulos[0]))
pprint(dfLimpio[0][178])

176
'u'
